In [71]:
import pandas as pd
import numpy as np
import scipy as sp
import collections
import gc
import pickle
import time
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Lambda, Flatten, Conv2DTranspose
from tensorflow.keras.layers import Dropout, GaussianNoise, Input, UpSampling2D
from tensorflow.keras.models import Model, Sequential, load_model
import sklearn.model_selection

<h2> Data Preprocessing </h2>

In [72]:
start = time.time()
data = pd.read_csv("./train.csv")
X1 = data[data.columns[np.vectorize(lambda s: "start" in s)(data.columns.values)]]
X1 = X1.values.reshape(data.shape[0],25,25,1)
delta = data["delta"].values
X2 = np.zeros((data.shape[0],25*25*5))
for i in range(data.shape[0]):
    X2[i,(delta[i]-1)::5] = 1
temp = data[data.columns[np.vectorize(lambda s: "stop" in s)(data.columns.values)]].values
Y = np.zeros((data.shape[0],25*25*5))
for i in range(data.shape[0]):
    Y[i,(delta[i]-1)::5] = temp[i]
print("Preprocessing Exited in "+str(time.time()-start)+" seconds")
data.shape, X1.shape, X2.shape, Y.shape

Preprocessing Exited in 8.937174081802368 seconds


((50000, 1252), (50000, 25, 25, 1), (50000, 3125), (50000, 3125))

<h2>Approach 1: N-Step Convolver

In [98]:
#posweight = (625-np.sum(Y,axis=1).mean())/np.sum(Y,axis=1).mean()
#Pre-fed value to speed computation
def weighted_mean_squared_error(ytrue, ypred, posweight=5.826105561517628):
    diff = ytrue-ypred
    poserr = tf.map_fn(lambda s: tf.reduce_sum(tf.math.square(s[s>0])), diff)*posweight
    negerr = tf.map_fn(lambda s: tf.reduce_sum(tf.math.square(s[s<0])), diff)
    return tf.reduce_mean(poserr)+tf.reduce_mean(negerr)

In [100]:
tf.keras.backend.clear_session()
deltainp = Input((25*25*5), dtype=tf.float32) #Delta Mask
gofinp = Input((25,25,1)) #Game of Life Board
#Increasing Filters to Capture Relationships
conv1 = Conv2D(filters=32, kernel_size=(7,7), strides=(1,1), padding="same", activation="relu")(gofinp)
max1 = MaxPooling2D(pool_size=(2,2), strides=(1,1), padding="same")(conv1)
conv2 = Conv2D(filters=64, kernel_size=(7,7), strides=(1,1), padding="same", activation="relu")(max1)
max2 = MaxPooling2D(pool_size=(2,2), strides=(1,1), padding="same")(conv2)
#Reducing Filters to Final Output
conv3 = Conv2D(filters=32, kernel_size=(7,7), strides=(1,1), padding="same", activation="relu")(max2)
max3 = MaxPooling2D(pool_size=(2,2), strides=(1,1), padding="same")(conv3)
final = Conv2D(filters=5, kernel_size=(7,7), strides=(1,1), padding="same", activation=None)(max3)
#Evaluate Required Board
final = tf.keras.layers.Activation(activation="sigmoid")(Flatten()(final))
final = Flatten()(tf.math.multiply(final, deltainp))
model = Model(inputs=[gofinp, deltainp], outputs=final)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-3), loss=weighted_mean_squared_error)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 25, 25, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 25, 25, 32)   1600        input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 25, 25, 32)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 25, 25, 64)   100416      max_pooling2d[0][0]              
______________________________________________________________________________________________

In [104]:
hist = model.fit([X1[:40000],X2[:40000]],Y[:40000], batch_size=256, epochs=1)
pd.Series(hist.history["loss"]).plot()
model.save("./40000example.h5")

Train on 40000 samples
40000/40000 [==============================] - 223s 6ms/sample - loss: 186.0795


<h2> Approach 2: CNN-Encoder-Decoder </h2>

In [153]:
tf.keras.backend.clear_session()
deltainp = Input((25,25,5), dtype=tf.float32) #Delta Mask
gofinp = Input((25,25,1)) #Game of Life Board
conv1 = Conv2D(filters=32, kernel_size=(5,5), strides=(1,1), padding="same", activation="relu")(gofinp)
max1 = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="valid")(conv1)
conv2 = Conv2D(filters=64, kernel_size=(5,5), strides=(1,1), padding="same", activation="relu")(max1)
max2 = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="valid")(conv2)
drp2 = Dropout(0.1)(max2)
conv3 = Conv2D(filters=128, kernel_size=(5,5), strides=(1,1), padding="same", activation="elu")(drp2)
max3 = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="valid")(conv3)
up1 = UpSampling2D(size=(2,2), interpolation="nearest")(max3)
dec1 = Conv2DTranspose(filters=64, strides=(1,1), kernel_size=(5,5), padding="same", activation="elu")(up1)
up2 = UpSampling2D(size=(2,2), interpolation="nearest")(dec1)
dec2 = Conv2DTranspose(filters=32, strides=(1,1), kernel_size=(5,5), padding="same", activation="relu")(up2)
up3 = UpSampling2D(size=(2,2), interpolation="nearest")(dec2)
dec3 = Conv2DTranspose(filters=5, strides=(1,1), kernel_size=(2,2), activation="sigmoid")(up3)
final = Flatten()(tf.math.multiply(dec3, deltainp))
model = Model(inputs=[gofinp, deltainp], outputs=final)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2), loss="mean_absolute_error")
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 25, 25, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 25, 25, 32)   832         input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 12, 12, 32)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 12, 12, 64)   51264       max_pooling2d[0][0]              
______________________________________________________________________________________________

<h2> Approach 3: Stacked Encoder with Data Augmentation